In [1]:
import data_extraction
import pandas as pd

pd.set_option('display.max_columns', None)

myObject = data_extraction.DataExtractor()
link = "https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf"
df = myObject.retrieve_pdf_data(link)

type(df)

pandas.core.frame.DataFrame

df is a dataframe that now contains the data from our table
These are the first & last few lines of the database then!

In [2]:
df.head(5)

,card_number,expiry_date,card_provider,date_payment_confirmed
0,30060773296197,09/26,Diners Club / Carte Blanche,2015-11-25
1,349624180933183,10/23,American Express,2001-06-18
2,3529023891650490,06/23,JCB 16 digit,2000-12-26
3,213142929492281,09/27,JCB 15 digit,2011-02-12
4,502067329974,10/25,Maestro,1997-03-13


In [3]:
df.tail(5)

,card_number,expiry_date,card_provider,date_payment_confirmed
15304,180036921556789,12/28,JCB 15 digit,1997-06-06
15305,180018030448512,11/24,JCB 15 digit,2004-06-16
15306,3569953313547220,04/24,JCB 16 digit,2020-02-05
15307,4444521712606810,06/27,VISA 16 digit,2008-06-16
15308,372031786522735,02/30,American Express,2009-02-04


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15309 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   card_number             15309 non-null  object
 1   expiry_date             15309 non-null  object
 2   card_provider           15309 non-null  object
 3   date_payment_confirmed  15309 non-null  object
dtypes: object(4)
memory usage: 478.5+ KB


In [5]:
df.dtypes

card_number               object
expiry_date               object
card_provider             object
date_payment_confirmed    object
dtype: object

card numberseems to be an integer

In [6]:
df['card_number'] = pd.to_numeric(df['card_number'],errors="coerce")

instead of just cast, ths one puts NAN if it's not a number

We have some date columns

In [7]:
df['expiry_date']=pd.to_datetime(df['expiry_date'],errors='coerce',format="%m/%y")
df['date_payment_confirmed']=pd.to_datetime(df['date_payment_confirmed'],infer_datetime_format=True, errors='coerce') 

inspect some potentially categorical data:
card_provider

In [8]:
df["card_provider"].describe()

count             15309
unique               25
top       VISA 16 digit
freq               2426
Name: card_provider, dtype: object

Not bad, only 25 card providers

In [9]:
df["card_provider"] = df["card_provider"].astype("category")


Nope. What about duplicate lines?

In [10]:
df.duplicated().sum()

10

ten! ok. Let's just check on the card related columns only - not order dates

In [11]:
df.duplicated(subset=["card_number",	"expiry_date",	"card_provider"	],keep="first").sum()

11

Still only 10, which is good I guess. Lets drop those

In [12]:
df.drop_duplicates(subset=["card_number",	"expiry_date",	"card_provider"	],keep="first",inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15298 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   card_number             15258 non-null  float64       
 1   expiry_date             15283 non-null  datetime64[ns]
 2   card_provider           15298 non-null  category      
 3   date_payment_confirmed  15283 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), float64(1)
memory usage: 493.7 KB


Looks like the date parsing failed for expiry date. Maybe we should fix that??

Date time format guessing over-ridden with string.
card data parsing also now improved.
let's drop nans

In [13]:
df.dropna(inplace=True,axis="index",how="any") # drops the whole row if any columns have an nan
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15258 entries, 0 to 15308
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   card_number             15258 non-null  float64       
 1   expiry_date             15258 non-null  datetime64[ns]
 2   card_provider           15258 non-null  category      
 3   date_payment_confirmed  15258 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](2), float64(1)
memory usage: 492.5 KB


Now put the code into a local database maybe

In [14]:
import database_utils


myObject = database_utils.DatabaseConnector()


myObject.upload_to_db(df,"dim_card_details")